# AI2023_C6_Project: Distribusi Strategis Tangki Air
Ketua: 
- Hugo Alfedo Putra (225150201111013)

Anggota:
- Farrel Rakha Dzakwan (NIM)
- Kartika Madania (NIM)
- Nafakhatul Fadhliyah (NIM)
- Rayhan Egar Sadtya N. (225150201111014)

## Project Description

- Detailed proposal: <a href="https://drive.google.com/file/d/17-KTcC_7lHK3ccReEwo3SYMl73sKyIs8/view?usp=sharing">Click Here (UB Account)</a>
- Case example: <a href="https://drive.google.com/file/d/1cw2pzo-zphH3vdL3rC1ZDHoAic22058L/view?usp=sharing">Click Here (UB Account)</a>

## Dependencies

In [17]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Class: Node
Node mereperesentasikan sebuah desa, memiliki atribut seperti:
- (str) name 
- (int[]) coordinate
- (int) population
- (object[]) neighbors

In [18]:
class Node:
    
    # initialization function
    def __init__(self, name, coordinate_x, coordinate_y, population):
        self.name = name
        self.coordinate = np.array([coordinate_x, coordinate_y])
        self.population = population
        self.neighbors = []
    
    # Establishes bi-directional path
    def add_neighbor(self, neighbor):
        # The if-condition below guarantees no recursive connections
        # i.e. a node can't be neighbors with itself
        if self.name != neighbor.name:
            sld = np.sqrt(np.power(neighbor.coordinate[0] - self.coordinate[0], 2) + np.power(neighbor.coordinate[1] - self.coordinate[1] ,2))
            cost = random.choice([i for i in range(100)])
            self.neighbors.append([neighbor, sld, cost])
            neighbor.neighbors.append([self, sld, cost])
    
    # Get neighbors
    def get_neighbor(self):
        its_neighbors = []
        for i in range(len(self.neighbors)):
            its_neighbors.append(self.neighbors[i][0].name)
        return its_neighbors
    
    def get_cost(self):
        its_costs = []
        for i in range(len(self.neighbors)):
            its_costs.append(self.neighbors[i][2])
        return its_costs
    
    def get_sld(self):
        its_slds = []
        for i in range(len(self.neighbors)):
            its_slds.append(self.neighbors[i][1])
        return its_slds

    # Print node information
    def print_info(self):
        print(f"Name: {self.name}")
        print(f"Coordinate [X, Y]: {self.coordinate}")
        print(f"Population: {self.population}")
        print(f"Neighbors: {self.get_neighbor()}")
        print(f"Cost: {self.get_cost()}")
        print(f"SLD: {self.get_sld()}")

    # Quick debug purposes
    def minimal_print(self):
        print(f"ID: {self.name} \t -> {self.get_neighbor()}")

## Class: Environment

In [19]:
class Environment:
    
    # Initialization function
    def __init__(self, name, area_size, node_number):
        self.name = name
        self.node_number = node_number
        self.empty_node_avail = node_number # kind of redundant (read more below)
        self.area_size = area_size
        self.area = [[None for _ in range(area_size)] for _ in range(area_size)]

        self.node_list = [] # Used to store generated nodes
        # Note: making an array filled with None and then *appending* instead of *updating*
        # the values doesn't make any sense because you'd have Nones in front of everything else
        
        self.node_score = [None for _ in range(node_number)] # Used to store f(n) values for generated nodes

    # Add node
    def add_node(self, node):
        self.area[node.coordinate[0]][node.coordinate[1]] = node
        self.node_list.append(node)
        self.empty_node_avail -= 1 # you have a while loop using "remaining" as a decreasing variable, I don't think this is useful
    
    # Generate test case environment
    def test_case(self):
        O_node = Node("O", 0, 0, 0)
        A_node = Node("A", -3, 3, 300)
        B_node = Node("B", 1, 6, 100)
        C_node = Node("C", 5, 5, 150)
        D_node = Node("D", 2, 3, 400)
        E_node = Node("E", 5, 3, 250)
        F_node = Node("F", 4, 0, 75)
        self.add_node(O_node)
        self.add_node(A_node)
        self.add_node(B_node)
        self.add_node(C_node)
        self.add_node(D_node)
        self.add_node(E_node)
        self.add_node(F_node)
        O_node.add_neighbor(D_node)
        O_node.add_neighbor(F_node)
        A_node.add_neighbor(B_node)
        A_node.add_neighbor(D_node)
        B_node.add_neighbor(A_node)
        B_node.add_neighbor(D_node)
        B_node.add_neighbor(C_node)
        C_node.add_neighbor(B_node)
        C_node.add_neighbor(E_node)
        D_node.add_neighbor(E_node)
        D_node.add_neighbor(F_node)
        self.node_number = 7

    # Populate area 
    def populate_area(self):
        # Makes sure the origin node is registered in node_list and area after creation
        new_node = Node("origin", 0, 0, 0)
        self.add_node(new_node)

        remaining = self.node_number
        naming_counter = 0

        # Generate randomly-placed nodes
        while(remaining > 0):

            # Randomize index
            x_loc = np.random.randint(1, self.area_size)
            y_loc = np.random.randint(1, self.area_size)

            # If index empty, initialize node
            if (self.area[x_loc][y_loc] is None):
                # Creates data for new_node
                node_name = str(self.name) + "node" + str(naming_counter) 
                naming_counter += 1
                node_population = np.random.randint(50, 1000)

                # Creates the new_node
                new_node = Node(node_name, x_loc, y_loc, node_population)

                # Adds the node using a function *you* have created before
                self.add_node(new_node)

                remaining -= 1

        # Generate randomly-put edges (neighbors) between nodes
        # Constraint: 
        #   1. Guaranteed edge from "origin" to at least one node
        #   2. Every node is reachable

        # choice is a list containing all possible integers from 0 to self.node_number
        choice = [i for i in range(self.node_number)]

        for i in range(len(self.node_list)-1):
            # Makes sure there is a path from origin to the last node where
            # it visits every other nodes on the way (satisfies all constraints)
            self.node_list[i].add_neighbor(self.node_list[i+1])

            # Connects one node with random neighbors if it passes a certain
            # threshold (chance > k)
            chance = np.random.random()
            if chance > 0.69: # change this if you want
                self.node_list[i].add_neighbor(self.node_list[random.choice(choice)])

### Environment Use Case

In [20]:
env = Environment("test", 16, 16)
env.test_case()
for i in range(env.node_number):
    env.node_list[i].minimal_print()

ID: O 	 -> ['D', 'F']
ID: A 	 -> ['B', 'D', 'B']
ID: B 	 -> ['A', 'A', 'D', 'C', 'C']
ID: C 	 -> ['B', 'B', 'E']
ID: D 	 -> ['O', 'A', 'B', 'E', 'F']
ID: E 	 -> ['C', 'D']
ID: F 	 -> ['O', 'D']


## Environment Evaluation

A* Search, expand minimum $f(n) = g(n) + h(n)$ node first.

$$ g(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} $$

$$ h(n) = \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$

$$ f(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} + \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$